PHYS 411

# Lab 1: Computer Exercises 

Kevin Kuo (V00788458)

In [1]:
import numpy as np
import sympy as sym
import scipy as sci
import matplotlib.pyplot as plt

from numpy import pi, cos
from sympy import symbols
from scipy.signal import welch 
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Power spectral density, periodogram

$$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$$

In [2]:
A = [250, 34, 8 , 10]

fn = 10
w0 = 2*pi*fn 
w1 = w0/100
w4 = w0/500
W = [w1, 4.3*w1, 4.7*w1, w4]

In [3]:
def t(NT): # Time
    t = np.linspace(0, NT/fn, NT)
    return t 

def CN(NT):
    CN = np.random.normal(0, 400**0.5, int(NT))
    return CN

def s(t): # The summation
    s = 0
    for i in range(len(A)-1):
        s += A[i] * cos(W[i]*t) 
    return s
                        
def S(NT): # Array of summation values
    S = s(t(NT))
    return S  

def x(NT): # Array of x(t) values
    x = CN(NT) + S(NT)
    return x

In [4]:
# Find the confidence interval:
df = 2
CI = 0.95
bounds = [(1-CI)/2, 1-(1-CI)/2]

chiT = df / (2 * sci.stats.chi2.ppf(bounds[0], df))
chiB = df / (2 * sci.stats.chi2.ppf(bounds[1], df))

In [5]:
NT = [20000, 2000]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
ax1.plot(t(NT[0]), x(NT[0]))
ax2.plot(t(NT[1]), x(NT[1]))
ax1.set_title('$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$ for time series length (NT) of 20,000 sampled at 10Hz', fontsize=14)
ax2.set_title('$x(t) = CN(t) + \sum_{i=1}^4 A_i \cos(\omega_i t)$ for time series length (NT) of 2000 sampled at 10Hz', fontsize=14)
ax1.set_xlabel('Time ($s$)', fontsize=12)
ax2.set_xlabel('Time ($s$)', fontsize=12)

fig.tight_layout()

<Figure size 864x720 with 2 Axes>

### Step-by-step calculation of PSD:

#### For NT = 20000

In [6]:
# Fourier transform: 
FT1 = np.fft.fft(x(NT[0]))
FS1 = np.fft.fftfreq(int(NT[0]), 1/fn)

In [7]:
# Plot the Fourier Transform :
fig, (axS1, axS2) = plt.subplots(2, 1, figsize = (10, 8))
axS1.plot(FS1, FT1.real)
axS2.plot(FS1, FT1.imag)

axS1.set_title('Real part of the Fourier transform of $x(t)$ ($Re [ F \{x(t)\} ]$) for time series of length 20,000')
axS2.set_title('Imaginary part of the Fourier transform of $x(t)$ ($Im [ F \{x(t)\} ]$) for time series of length 20,000')

axS1.set_xlabel('Frequency (Hz)')
axS2.set_xlabel('Frequency (Hz)')

axS1.set_ylabel('$Re[X(f)]$')
axS2.set_ylabel('$Im[X(f)]$')

fig.tight_layout()

<Figure size 720x576 with 2 Axes>

In [8]:
# Plot the 1-sided spectral density function of the Fourier transform 
# fig, (axG1, axG2) = plt.subplots(2, 1, figsize = (10, 8))
# axG1.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].real))
# axG2.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].imag))

# axG1.set_title('Real part of the one-sided power spectral density function of $x(t)$ ($Re[G_{xx}(f)]$) for time series of length 20,000')
# axG2.set_title('Imaginary part of the one-sided power spectral density function of $x(t)$ ($Im[G_{xx}(f)]$) for time series of length 20,000')

# axG1.set_xlabel('Frequency (Hz)')
# axG2.set_xlabel('Frequency (Hz)')

# axG1.set_ylabel('$Re[G_{xx}(f)]$')
# axG2.set_ylabel('$Im[G_{xx}(f)]$')

# fig.tight_layout()

In [9]:
fig, (axG3, axG4) = plt.subplots(2, 1, figsize = (10, 8))
# fig, (axG3, axG4, axG5, axG6) = plt.subplots(4, 1, figsize = (10, 16))
axG3.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].real) * chiT * 4/3, '--', alpha = 0.5, color = 'b')
axG3.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].real) * chiB * 4/3, '--', alpha = 0.5, color = 'b')
axG3.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].real))

axG4.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].imag) * chiT * 4/3, '--', alpha = 0.5, color = 'b')
axG4.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].imag) * chiB * 4/3, '--', alpha = 0.5, color = 'b')
axG4.loglog(FS1[0:1000], np.abs(4*FT1[0:1000].imag))

# axG5.loglog(FS1[0:100], np.abs(4*FT1[0:100].real) * chiT * 4/3, '--', alpha = 0.5, color = 'b')
# axG5.loglog(FS1[0:100], np.abs(4*FT1[0:100].real) * chiB * 4/3, '--', alpha = 0.5, color = 'b')
# axG5.loglog(FS1[0:100], np.abs(4*FT1[0:100].real))

# axG6.loglog(FS1[0:100], np.abs(4*FT1[0:100].imag) * chiT * 4/3, '--', alpha = 0.5, color = 'b')
# axG6.loglog(FS1[0:100], np.abs(4*FT1[0:100].imag) * chiB * 4/3, '--', alpha = 0.5, color = 'b')
# axG6.loglog(FS1[0:100], np.abs(4*FT1[0:100].imag))

title1 = 'Real part of the one-sided power spectral density function of $x(t)$ ($Re[G_{xx}(f)]$) for time series of length 20,000, with 95% confidence interval'
title2 = 'Imaginary part of the one-sided power spectral density function of $x(t)$ ($Im[G_{xx}(f)]$) for time series of length 20,000, with 95% confidence interval'
# title3 = 'Real part of the one-sided power spectral density function of $x(t)$ ($Re[G_{xx}(f)]$) for time series of length 2000, with 95% confidence interval'
# title4 = 'Imaginary part of the one-sided power spectral density function of $x(t)$ ($Im[G_{xx}(f)]$) for time series of length 2000, with 95% confidence interval'

axG3.set_title("\n".join(wrap(title1, 100)))
axG4.set_title("\n".join(wrap(title2, 100)))
# axG5.set_title("\n".join(wrap(title3, 100)))
# axG6.set_title("\n".join(wrap(title4, 100)))

axG3.set_xlabel('Frequency (Hz)')
axG4.set_xlabel('Frequency (Hz)')
# axG5.set_xlabel('Frequency (Hz)')
# axG6.set_xlabel('Frequency (Hz)')

axG3.set_ylabel(r'$Re[G_{xx}(f)]$ [$\frac{V^2}{Hz}$]')
axG4.set_ylabel(r'$Im[G_{xx}(f)]$ [$\frac{V^2}{Hz}$]')
# axG5.set_ylabel(r'$Re[G_{xx}(f)]$ [$\frac{V^2}{Hz}$]')
# axG6.set_ylabel(r'$Im[G_{xx}(f)]$ [$\frac{V^2}{Hz}$]')

fig.tight_layout()

<Figure size 720x576 with 2 Axes>

#### For NT = 2000

In [10]:
# Fourier transform: 
FT2 = np.fft.fft(x(NT[1]))
FS2 = np.fft.fftfreq(int(NT[1]), 1/fn)

In [11]:
# Plot the Fourier transform:
fig, (axS3, axS4) = plt.subplots(2, 1, figsize = (10, 8))
axS3.plot(FS2, FT2.real)
axS4.plot(FS2, FT2.imag)
axS3.set_title('Real part of the Fourier transform of $x(t)$ ($Re [ F \{x(t)\} ]$) for time series of length 2000')
axS4.set_title('Imaginary part of the Fourier transform of $x(t)$ ($Im [ F \{x(t)\} ]$) for time series of length 2000')

axS3.set_xlabel('Frequency (Hz)')
axS4.set_xlabel('Frequency (Hz)')

axS3.set_ylabel('$Re[X(f)]$')
axS4.set_ylabel('$Im[X(f)]$')

fig.tight_layout()

<Figure size 720x576 with 2 Axes>

In [12]:
# Plot the 1-sided spectral density function of the Fourier transform 
# fig, (axG5, axG6) = plt.subplots(2, 1, figsize = (10, 8))
# axG5.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].real))
# axG6.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].imag))
# axG5.set_title('Real part of the one-sided power spectral density function of $x(t)$ ($Re[G_{xx}(f)]$) for time series of length 2000')
# axG6.set_title('Imaginary part of the one-sided power spectral density function of $x(t)$ ($Im[G_{xx}(f)]$) for time series of length 2000')

# axG5.set_xlabel('Frequency (Hz)')
# axG6.set_xlabel('Frequency (Hz)')

# axG5.set_ylabel('$Re[G_{xx}(f)]$')
# axG6.set_ylabel('$Im[G_{xx}(f)]$')

# fig.tight_layout()

In [13]:
fig, (axG7, axG8) = plt.subplots(2, 1, figsize = (10, 8))
axG7.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].real) * chiT * 4/3, '--', alpha = 0.5, color = 'b')
axG7.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].real) * chiB * 4/3, '--', alpha = 0.5, color = 'b')
axG7.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].real))

axG8.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].imag) * chiT * 4/3, '--', alpha = 0.5, color = 'b')
axG8.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].imag) * chiB * 4/3, '--', alpha = 0.5, color = 'b')
axG8.loglog(FS2[0:1000], np.abs(4*FT2[0:1000].imag))

title3 = 'Real part of the one-sided power spectral density function of $x(t)$ ($Re[G_{xx}(f)]$) for time series of length 2000, with 95% confidence interval'
title4 = 'Imaginary part of the one-sided power spectral density function of $x(t)$ ($Im[G_{xx}(f)]$) for time series of length 2000, with 95% confidence interval'

axG7.set_title("\n".join(wrap(title3, 100)))
axG8.set_title("\n".join(wrap(title4, 100)))

axG7.set_xlabel('Frequency (Hz)')
axG8.set_xlabel('Frequency (Hz)')

axG7.set_ylabel(r'$Re[G_{xx}(f)]$ [$\frac{V^2}{Hz}$]')
axG8.set_ylabel(r'$Im[G_{xx}(f)]$ [$\frac{V^2}{Hz}$]')

fig.tight_layout()

<Figure size 720x576 with 2 Axes>

#### Questions: 

##### 1) Does increasing the record length improve the estimate? 

No. The estimate is based on the chi-squared distribution and the number of sub-spectras taken, hence incerasing record length will have no affect the accuracy nor confidence level of the estimate. 

##### 2) How does the record length affect the frequency resolution?

From the Frourier transforms of the two record lengths, increasing the record length has caused the peaks to become more pronounced. It makes sense as the increase in record length increases the amount of samples, which decreases the affect of noise while increasing the amount of cycles, hence increasing the height of the peaks.  

### Use built in function of pwelch (well...the Python equivalent):

#### In MatLab: 

`\[pxx, f\] = pwelch(x, window, noverlap, f, fs)`

"Returns a frequency vector in cylcles per unit time" (via MatLab documentation) 

> pxx = Power spectral density 

> f = Cyclical frequency 

> window = Tapered window (default = Hanning) 

> noverlap = Amount of overlapping datapoints

> f = Frequency 

> fs = Sample rate

**Let:**

> xx1 = Time series data points 

> NFFT = Number of data points used in the fast Fourier transform 

> fsamp = Sampling frequency of the time series 


**This means:** `\[PP_M1, fq_M1\] = pwelch(xx1, NFFT1, NFFT1/2, NFFT1, fsamp, 'onesided');`

> window = NFFT1 

> noverlap = NFFT1/2

> f = NFFT1

> 'onesided' =  Returns onesided power spectral density of x(t) 


#### In Python: 

`scipy.signal.welch(x, fs=1.0, window='hanning', nperseg=256, noverlap=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)`

Returns power spectral density (or power spectrum) of x(t), and array of sample fequencies

> x = Time series 

> fs = Sampling frequency (default = 1 Hz)

> window = Tapered window (default = Hanning) 

> nperseg = Length of segments (default = 256) 

> noverlap = Amount of overlapping datapoints (default = None) 

> nfft = Length of the fast Fourier transform 

> detrend = Options to detrend each segment (default = constant) 

> return_onesided = Option to return onesided function or not (default = True) 

> scaling = Option to select between computing the power spectral density, or the power spectrum (default = density) 

> axis = Axis used to compute the periodogram (default = -1 the last axis) 

In [14]:
RL = len(x(NT[0]))
NSize = np.array([1/(2**i) for i in range(1, 5)])
NFFTs = (NSize*RL).astype(int)

In [17]:
def Welch2(i):
    return welch(x(NT[0]), 10, nperseg=NFFTs[i], \
                window = sci.signal.windows.hann(int(NFFTs[i])), \
                noverlap = NFFTs[i]/2, nfft = NFFTs[i], detrend = False, \
                return_onesided = True, scaling = 'spectrum')[1]

title5 = 'One-sided power spectral density function of $x(t)$, with 95% confidence interval'
for i in range(len(NFFTs)):
    ps1 = 0
    psu = 0
    psd = 0
    
    plt.figure(figsize = (10, 4))
    ps1 = np.array(Welch2(i))
    psu = ps1 * NSize[i] * 4/3 * chiT
    psd = ps1 * 4/3 * chiB
    plt.loglog(psu, linestyle = '--', alpha = 0.5, color = 'b')
    plt.loglog(psd, linestyle = '--', alpha = 0.5, color = 'b')
    plt.loglog(ps1, label=r'NFFT = 2^{0}'.format((int(np.log2(NFFTs[i])))))
    plt.title("\n".join(wrap(title5, 100)))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
    plt.legend()
    

<Figure size 720x288 with 1 Axes>

<Figure size 720x288 with 1 Axes>

<Figure size 720x288 with 1 Axes>

<Figure size 720x288 with 1 Axes>

As we can see from the graphs, the confidence interval decreases with the decrease in frequency resolution. Choosing a best NFFT would be an balancing act between preserving the resolution and confidence interval. Visually, it appears the best choice is the order of two closesd to half of the data points for time series, as it perserves most of the resolution while possessing an somewhat acceptible confidence intervals. Choosing an order of two close to a quarter of the data doesn't look too bad either.